In [ ]:
from exm.stitching.tileset import Tileset
from exm.utils import display_img as di 
import numpy as np

masks_path = '/mp/nas2/mansour/20221017_alignment_masks/20221017_alignment_slice1_masks.h5'
stitching_result = '/mp/nas3/fixstars/yves/zebrafish_data/20221017_alignment/stitching/slice1.xml'
output_path = "/tmp/"

ts=Tileset([0.1625, 0.1625, 0.4])

### Reconstruct a stitched segmented dataset

In [ ]:
# Initialize from a non BDV file format. The H5 file only contain tiles, we need to initialize the offsets from 
# another source

segfile = f"{masks_path}"
ts.init_from_h5(segfile, downscale=[5,5,2], progress=True)

In [ ]:
# To initialize offsets we use the XML file obtained from the stitching step /tmp/ if you went through the steps 
# of the previous notebook. Here we initialize from a known good result

stitchfile = f"{stitching_result}"
ts.update_offsets(stitchfile)

In [ ]:
# Check if this looks good. 

di((ts.show_slice(51).T%256).astype(np.uint8))

# Notice that some nuclei on the seams have multiple colors, we will fix that with the deduplication

### Deduplicate nuclei IDs

In [ ]:
_ = ts.dedup_segmentation_ids(progress=True)

In [ ]:
di((ts.show_slice(51).T%256).astype(np.uint8))

### Find centroids

In [ ]:
centers = ts.get_centroids()

In [ ]:
centers

### Local to global transformations

Let's demonstrate how to transform an array of pipxel coordinates in the coordinate system of a tile into a global µm coordinate

In [ ]:
# This is a function to draw a 2D cross in a numpy image
# TODO: add it to utils maybe?

def draw_2d_cross(img, coords, color=255):
    x = coords[0]
    y = coords[1]
    img[:,y-1:y+2,x-4:x+5]=0
    img[:,y-4:y+5,x-1:x+2]=0
    img[:,y,x-4:x+5]=color
    img[:,y-4:y+5,x]=color

In [ ]:
# Here local coordinates are in pixel, the first coordinate indicates the tile number they belong to (here #5)

local_coords = np.array([[5, 280,160, 50], 
                         [5, 300,100, 50]], dtype=float)

In [ ]:
# Show the points in the individual tile 

a=np.copy(ts.tiles[5].img)
for c in coords[:,1:3]:
    draw_2d_cross(a,c.astype(int))
di(a[50])

In [ ]:
# this multiplier allows the conversion from our images pixels into µm
vox_size = np.array(ts.voxel_size)*np.array(ts.original_xyz_size)/np.array(ts[0].img.shape)[[2,1,0]]

In [ ]:
local_coords[:,1:] = local_coords[:,1:]*vox_size

In [ ]:
global_coords=ts.local_to_global(local_coords)

In [ ]:
b = ts.produce_output_volume()

In [ ]:
for rr in r:
    draw_2d_cross(b, (rr/vox_size).astype(int)[:2])

In [ ]:
# global_coords, once transformed into global pixel coordinates by dividing them by vox_size, show in the correct
# place in the reconstructed volume

di(b[50])